# TP6 - Page Rank

Dans ce TP, nous allons étudier comment implémenter une version simple de l'algorithme Page Rank, proposé par les concepteurs du moteur de recherche Google pour évaluer l'importance des pages web.

Nous vous rappelons que cet algorithme repose sur le calcul de la distribution stationnaire d'une chaîne de Markov irréductible et apériodique.

Pour obtenir un algorithme efficace, nous verrons comment utiliser le fait que les graphes manipulés sont peu denses (sparse en anglais).

Ce TP se divise en trois parties :  
1. étudier le calcul de la distribution stationnaire dans une chaîne de Markov.
2. implémenter une version de l'algorithme Page Rank pour des graphes dirigés, et l'évaluer sur des graphes aléatoires produits à l'aide des algorithmes vus au TP précédent (Erdos-Renyi, Barabasi-Albert).
3. étudier une solution visant à modifier le score obtenu par Page Rank, à l'aide d'une ferme de liens.

## Distribution stationnaire d'une chaîne de Markov

Pour cette partie, on suppose qu'on manipule une matrice carrée, représentée en Python comme une liste de listes, de dimension $n$. On suppose de plus que la matrice est stochastique, c'est-à-dire que pour chaque ligne, la somme des valeurs est égale à 1. 

De plus, on manipulera des vecteurs de même dimension que la matrice.

1. Ecrire une fonction qui réalise le produit d'un vecteur (ligne) $X$ avec une matrice $M$, c'est-à-dire le produit $X.M$. On rappelle que le résultat est un vecteur (ligne) $Y$ de même dimension que $X$, et dont la $j$-ème case
contient la valeur $\sum_{i=0}^{n-1} X_i * M_{i,j}$. 

In [6]:
def produit_vec_mat(X,M):
    # x : vecteur ligne de dimension n
    # M : matrice carrée stochastique de dimension n
    # retourne un vecteur ligne de dimension n, correspondant à X.M
    # <------------------------------------------------------------>
    
    i = 0 
    result = []
    Sum = 0.0
    i = 0
    for i in range(len(M)):
        for x in range(len(X)):
            Sum = Sum + (X[x] * M[x][i])
        result.append(Sum)
        i = i + 1
        Sum = 0
    return result

Vous pouvez tester votre fonction avec l'exemple suivant, vous devez retrouver les valeurs données dans les transparents du cours dernier (slide 42).

In [7]:
M = [
        [.6,.1,.3],
        [.6,.4,0],
        [.7,0,.3]
]

X = [.7,.3,0]
print(produit_vec_mat(X,M))
# Attendu : [.6, .19, .21]

[0.6, 0.19, 0.21]


2. On suppose dans la suite que la chaîne de Markov représentée par $M$ satisfait les conditions d'application du théorème assurant l'existence d'une unique distribution stationnaire, c'est-à-dire que la chaîne est bien irréductible et apériodique (voir les slides pour des détails). Un algorithme possible pour calculer cette distribution, appelée méthode des puissances, consiste à calculer les termes de la suite $(X_i)_{i\in \mathbb{N}}$ suivante :

$$\left \{ \begin{array}{ll}
X_0 &= [\frac1n,\frac1n,\ldots,\frac1n]\\
X_{i+1} &= X_{i}.M
\end{array} \right .$$

Ecrire une fonction qui prend en argument la matrice $M$, un nombre de pas $N$, et un seuil de précision $\epsilon$,
et qui effectue le calcul de la suite précédente jusqu'à ce que deux termes successifs soient à distance au plus $\epsilon$, ou, si cela n'arrive pas, jusqu'au $N$-ème terme de la suite. On précise que la distance entre les deux termes correspond ici à la norme 1, c'est-à-dire la somme des valeurs absolues des différences des composantes. Formellement, la distance entre deux vecteurs $X$ et $Y$ est égale à $d(X,Y) = \sum_{i=0}^{n-1}|X_i-Y_i|$. Vous pouvez utiliser la fonction `abs` de Python qui calcule la valeur absolue d'un nombre réel.

In [8]:
def distance(X,Y):
    d = sum([abs(X[i] - Y[i]) for i in range(len(X))])
    return d
def calcul_distrib_stat(M,N,eps):
    # M : matrice carrée stochastique de dimension n
    # On suppose en plus que la chaîne associée est irréductible et apériodique
    # N : nombre de pas
    # eps : précision (flottant)
    X = []
    for i in range(len(M)):
        X.append(1/len(M))
    for i in range(N):
        Y = produit_vec_mat(X,M)
        if distance(X,Y) < eps:
            return Y
        X = Y
    return   X 

A nouveau, comparez le résultat obtenu par votre fonction avec ceux des transparents du cours (slide 44).

In [9]:
stat = calcul_distrib_stat(M,10,10**(-5))
print(stat)
# Attendu : [.62,.1,.27]] (aux approximations près)

[0.6268634276999998, 0.10448836329999998, 0.26864820899999997]


3. Retour sur le produit matriciel. La complexité du produit matriciel implémenté est en $O(n^2)$, où $n$ indique la dimension de la matrice.
Par conséquent, dans le cas où la matrice $M$ est peu dense, c'est-à-dire que beaucoup de cases sont égales à $0$, il
est plus intéressant d'obtenir un algorithme linéaire dans le nombre de cases non vide, c'est-à-dire dans le nombre d'arêtes du graphe.
Pour cela, on suppose à présent que la matrice $M$ est représentée comme un dictionnaire de dictionnaires. Plus précisément, on
associe à chaque sommet un dictionnaire, qui donne, pour chaque sommet cible, le poids de la transition correspondante. Seules les transitions de poids non nul sont indiquées dans ce dictionnaire. On suppose ici que les sommets sont numérotés de 0 à n-1. Un exemple de cette représentation est donné ci-dessous, correspondant à la matrice précédente :

In [10]:
sparseM = {
    0 : {0:.6, 1:.1, 2:.3},
    1 : {0:.6, 1:.4},
    2 : {0:.7, 2:.3}
}
M = [
        [.6,.1,.3],
        [.6,.4,0],
        [.7,0,.3]
]
for source in sparseM:
    for target in sparseM[source]:
        print(source,target,sparseM[source][target])

0 0 0.6
0 1 0.1
0 2 0.3
1 0 0.6
1 1 0.4
2 0 0.7
2 2 0.3


Ecrire un algorithme pour le calcul du produit matriciel $X.M$ utilisant cette représentation.
Vous vérifierez votre résultat avec l'exemple précédent.

In [11]:
        
def produit_vec_mat_sparse(X,M):
    # x : vecteur ligne de dimension n
    # M : dictionnaire de dictionnaire représentant la matrice stochastique de dimension n
    # retourne un vecteur ligne de dimension n, correspondant à X.M
    Y = [0] * len(X)
    for source in M:
        for target in M[source]:
            Y[target] += X[source] * M[source][target]
    return Y

IndentationError: expected an indented block (<ipython-input-11-38df786127ad>, line 8)

In [12]:
print(produit_vec_mat(X,M))
print(produit_vec_mat_sparse(X,sparseM))
# Attendu : [.6, .19, .21]

[0.6, 0.19, 0.21]


NameError: name 'produit_vec_mat_sparse' is not defined

## Implémentation de Page Rank

Dans cette partie, on va travailler avec des graphes produits à l'aide de la librairie 'networkx'.

On rappelle que l'algorithme de Page Rank peut être résumé de la façon suivante :
1. On construit à partir du graphe la matrice stochastique $M$ dans laquelle les poids sont obtenus en mettant une probabilité uniforme sur les transitions sortantes.
2. On modifie cette matrice en tenant compte des puits (saut uniforme sur tous les sommets du graphe), on obtient ainsi la matrice $M_2$. On ajoute ensuite une petite probabilité de saut arbitraire, afin d'avoir une chaîne de Markov irréductible et apériodique. On obtient ainsi la matrice $P$, appelée matrice Page Rank.
3. On calcule la distribution stationnaire associée à $P$.

1. Pour la première étape, nous allons utiliser une fonction existante de `networkx`, intitulée `stochastic_graph`.

Tester cette fonction à l'aide du code suivant. Quel est le format utilisé pour représenter la matrice stochastique ? Comment sont traités les noeuds puits ?

In [13]:
import networkx as nx
import matplotlib.pyplot as plt

# Def du graphe
G = nx.DiGraph()
G.add_edge(0,1)
G.add_edge(1,0)
G.add_edge(0,2)
G.add_edge(1,3)
G.add_edge(3,0)

# Matrice stochastique associée
M = nx.stochastic_graph(G)
for x in M:
    print(x,":",M[x])

# Affichage du graphe
plt.figure(figsize=(15,9))
nx.draw(G,with_labels = True)
plt.show()

ModuleNotFoundError: No module named 'networkx'

2. Pour identifier les noeuds puits, nous allons utiliser la fonction `out_degreee()` de la librairie `networkx`.
Regardez la documentation pour voir comment s'en servir.

3. Pour la deuxième étape, la matrice $M$ obtenue à la première étape est modifiée au niveau des noeuds puits
en ajoutant la possibilité d'aller vers tous les sommets du graphe (avec proba uniforme), donnant ainsi la matrice $M_2$. Ensuite, la matrice Page Rank $P$ est définie comme suit :
$$
P = \alpha.M_2 + (1-\alpha).J
$$
où $\alpha$ est un coefficient choisi entre 0.8 et 0.85, et $J$ est la matrice associant $\frac1n$ à toutes les cases.

Dans la suite, étant donné un vecteur (ligne) $X$, on souhaite calculer efficacement le produit $Y = X.P$. Pour cela,  on note $A$
la somme des valeurs de $X$ correspondant à des noeuds puits, c'est-à-dire :
$$
A = \sum\{X_i\ \mid \ i \text{ est un puits} \}
$$

Alors on peut montrer que, pour tout indice $j$, on a : (voir plus bas pour une justification)

$$
Y_j = \alpha.(X.M)_j + \frac{\alpha}{n}.A + \frac{1-\alpha}{n}
$$

En d'autres termes, il suffit de faire le calcul du produit matriciel (sparse) de $X$ par $M$ (cf Question 3 de la partie 1),
puis d'ajouter à tous les coefficients la constante $\frac{\alpha}{n}.A + \frac{1-\alpha}{n}$. Attention, on a bien utilisé ici la matrice $M$, et pas la matrice $M_2$.

A partir de ces observations, écrire une fonction qui prend en entrée un graphe, la valeur du coefficient alpha, un nombre de pas, et une précision, et qui retourne
les valeurs de Page Rank obtenues selon cette méthode :

In [22]:
def PageRank(G,alpha,N,eps):
        # G : DiGraph Networkx
        # alpha : valeur du coefficient de la matrice de Page Rank (choisi dans [0.8,0.85])
        # N : nombre de pas max du calcul itératif (méthode des puissances)
        # eps : précision (flottant)
        M = nx.stochastic_graph(G)
        n= M.number_of_nodes()
        X = [1/n]*n
        nodes = list(G.nodes)
        puits = []
        for node in nodes:
            if(G.out_degree(node) == 0):
                puits.append(node)
        for i in range(N):
            A = sum([X[puit] for puit in puits])
            Y = produit_vec_mat_sparse(X,M)
            for j in range(n):
                Y[j] += (alpha * Y[j]) + ((alpha /n) * A) + ((1 - alpha) / n)
           
            if(dist(X,Y)) < eps:
                return Y
            X = Y
        return Y

3. Appliquer votre fonction à des graphes générés par Erdos-Renyi, Barabasi-Albert, Watts-Strogatz. Vous prendrez des graphes avec environ 200 sommets, et un degré moyen de l'ordre de 20.
Affichez-les résultats sous la forme d'histogrames. Ces résultats sont-ils cohérents avec l'étude faite de ces graphes au TP précédent ?

4. Si vous êtes dans les temps, comparez votre implémentation et celle de networkx (`nx.pagerank`).

## Fermes de liens

On rappelle ici le principe de la ferme de liens, présenté en cours (voir slides 54 et suivants). L'objectif est d'améliorer le score Page Rank obtenu par une page web. Pour cela, des pages inutiles, que l'on contrôle, sont ajoutées, et pointées par et vers la page dont on veut améliorer le score.

1. Nous allons travailler avec un graphe obtenue par l'algorithme de Barabasi-Albert, possédant 200 noeuds, et un degré moyen de 20. Ecrire un programme permettant de créer un tel graphe. Afficher l'histogramme des scores Page Rank associé.

2. Ecrire une fonction qui prend en argument un graphe, et retourne un tuple de deux éléments donnant le sommet dont le score Page Rank est minimal, avec son score Page Rank.

In [14]:
def compute_min_PR(G):
    # G : DiGraph

    return

3. Ecrire une fonction qui prend en entrée un graphe $G$, l'indice $i$ du sommet dont on veut modifier le score, et un entier $K$ indiquant le nombre de pages web dans la ferme de liens, et qui modifie le graphe $G$ de sorte à créer la ferme de liens comme indiqué plus haut.

In [15]:
def construire_ferme(G,i,k):
    # G : graphe orienté
    # i : un sommet de G
    # K : un entier, nombre de pages de la ferme à ajouter à G, reliées à i
            
    return

4. Nous allons étudier l'effet de l'ajout de la ferme de liens de façon empirique. Pour cela, nous allons suivre les étapes suivantes :
- construire un graphe de Barabasi-Albert
- identifier le sommet $i$ de score minimal
- ajouter la ferme de liens de taille $K$
- mesurer le nouveau score Page Rank du sommet $i$


Ecrire un programme qui réalise les étapes précédentes, pour un même graphe, en faisant varier $K$, de sorte à pouvoir tracer une courbe montrant l'évolution du nouveau score de $i$ en fonction de $K$.

## Annexes

Détail des calculs du produit matriciel pour la matrice Page Rank.

La matrice Page Rank est définie comme suit :
$$
P = \alpha.M + (1-\alpha).J
$$
où $M$ est la matrice stochasique obtenue précédemment, $\alpha$ est un coefficient choisi entre 0.8 et 0.85, et $J$ est la matrice associant $\frac1n$ à toutes les cases.

A l'aide de cette formule, et étant donné un vecteur (ligne) $X$, on peut réécrire le produit $Y = X.P$. Le coefficient $j$ peut s'écrire comme suit :
$$
\begin{array}{lll}
Y_j & =& \sum_{i=0}^{n-1} X_i * P_{i,j}\\
    & =& \sum_{i=0}^{n-1} X_i * (\alpha.M_{i,j} + (1-\alpha).J_{i,j})\\
    & =& \alpha \sum_{i=0}^{n-1} (X_i * M_{i,j}) + (1-\alpha).\sum_{i=0}^{n-1} (X_i. J_{i,j})\\
    & =& \alpha \sum_{i \mid \text{non puits}} (X_i * M_{i,j}) + \alpha \sum_{i \mid \text{puits}} (X_i * M_{i,j}) + \frac{1-\alpha}{n}.\sum_{i=0}^{n-1} X_i\\
    & =& \alpha (X.M')_j + \alpha \sum_{i \mid \text{puits}} (X_i * \frac1n ) + \frac{1-\alpha}{n}\\
    & =& \alpha (X.M')_j + \frac{\alpha}{n} \sum_{i \mid \text{puits}} (X_i) + \frac{1-\alpha}{n}\\
    & =& \alpha (X.M')_j + \frac{\alpha}{n} A + \frac{1-\alpha}{n}
\end{array}
$$